In [ ]:
!pip install wandb -qqq
!apt install tree

     |████████████████████████████████| 1.9 MB 10.8 MB/s 
     |████████████████████████████████| 182 kB 53.1 MB/s 
     |████████████████████████████████| 173 kB 64.1 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 168 kB 59.5 MB/s 
     |████████████████████████████████| 168 kB 65.6 MB/s 
     |████████████████████████████████| 166 kB 71.9 MB/s 
     |████████████████████████████████| 166 kB 69.5 MB/s 
     |████████████████████████████████| 162 kB 71.4 MB/s 
     |████████████████████████████████| 162 kB 75.6 MB/s 
     |████████████████████████████████| 158 kB 74.7 MB/s 
     |████████████████████████████████| 157 kB 76.5 MB/s 
     |████████████████████████████████| 157 kB 80.7 MB/s 
     |████████████████████████████████| 157 kB 76.5 MB/s 
     |████████████████████████████████| 157 kB 51.8 MB/s 
     |████████████████████████████████| 157 kB 75.3 MB/s 
     |████████████████████████████████| 157 kB 81.3 MB/s 
     |██████████

In [ ]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import random
import torch
import torchvision
from torch.utils.data import TensorDataset
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from math import sqrt
import math, time
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error
from torch.utils.data import DataLoader

## BASELINE MODEL

In [ ]:
#in case of need, convert date to datetime
def preprocess_baseline(data):
    """
    ## Prepare the data
    """
    data['Date'] = pd.to_datetime(data.Date)
    return data

In [ ]:
#just create basic model to use this function as a model
def model_persistence(x):
    return x

The same splitting function was applied for baseline and LSTM to evaluate the forecast with the same splits.

In [ ]:
#split the dataset to create test, validation and training dataset
def split(df, SeqLen): #splitdataset
  split = [int(len(df)*0.8), int(len(df)*0.9)] #give split index for train, valid, test
  train = df[:split[0]]  #~%80
  val = df[split[0]-SeqLen:split[1]] # ~%10
  test = df[split[1]-SeqLen:]  # ~%10
  return train, val, test

SeqLen = 5 #give sequence lenght for splitting

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error

def baseline_model_create(data):

  data=data.set_index('Date')
  values = pd.DataFrame(data[["Price"]])

  #shift the stock prices to use previous days stock prices for today
  dataframe = pd.concat([values.shift(1), values], axis=1) 
  dataframe.columns = ['t-1', 't']
  X = dataframe.values

  #get train, test and validation datasets
  train, valid, test = split(X,SeqLen) 

  train_X, train_y = train[:,0], train[:,1]
  valid_X, valid_y = valid[:,0], valid[:,1]
  test_X, test_y = test[:,0], test[:,1]

  predictions = list()
  for x in test_X:
    yhat = model_persistence(x)
    predictions.append(yhat)  #create data for the predictions
  mse_test_score = mean_squared_error(test_y, predictions) #calculate mse test error
  mape_test_score = mean_absolute_percentage_error(test_y, predictions) #calculate mape test error
  return mse_test_score ,mape_test_score


The baseline model application was applied to 3 different companies. To reach different datasets, 3 different functions were created. The other reason to create 3 different functions, the datasets can have different splits according to the applications and studies. For example, the range of the dataset Apple was changed because of the poor model performance issues.

**APPLE**

In [ ]:
#log the baseline model MSE and MAPE test scores to the W&B platform
def baseline_model_apple():

    with wandb.init(project="DSS") as r:
        
        
        # the last version of the Apple dataset is used
        artifact = r.use_artifact('metu_datacraft/DSS/Apple:v2', type='Data')
        table = artifact.get('weekday_data')
        dataset= {"Date": table.get_column("Date"),"Price":table.get_column("Price"),"Open":table.get_column("Open"),
                  "High":table.get_column("High"), "Low":table.get_column("Low") , "Volume": table.get_column("Volume"),
                  "Change":table.get_column("Change")}

        data = pd.DataFrame(dataset)
        data['Date'] = pd.to_datetime(data.Date)

        #the data was filtered to get the values of stock prices after the year 2020 to get more powerful model 
        data = data[data['Date']>'2020-01-01']
        data.reset_index(inplace=True)
        data.drop(columns={'index'},inplace=True)

        preprocessed_data=preprocess_baseline(data)
        mse_test_score,mape_test_score=baseline_model_create(preprocessed_data)

        wandb.init(project="DSS",job_type="baseline-model", name="baseline_model_APPLE")
        wandb.log({"mse_test_score": mse_test_score, "mape_test_score": mape_test_score}) #log the scores
      
baseline_model_apple()

**SAMSUNG**

In [ ]:
def baseline_model_samsung():

    with wandb.init(project="DSS") as r:
         
        
        # the last version of the Samsung dataset is used
        artifact = r.use_artifact('metu_datacraft/DSS/Samsung:v2', type='Data')
        table = artifact.get('weekday_data')
        dataset= {"Date": table.get_column("Date"),"Price":table.get_column("Price"),"Open":table.get_column("Open"),
                  "High":table.get_column("High"), "Low":table.get_column("Low") , "Volume": table.get_column("Volume"),
                  "Change":table.get_column("Change")}

        data = pd.DataFrame(dataset)
        preprocessed_data=preprocess_baseline(data)
        mse_test_score,mape_test_score=baseline_model_create(preprocessed_data)

        wandb.init(project="DSS",job_type="baseline-model", name="baseline_model_SAMSUNG")
        wandb.log({"mse_test_score": mse_test_score, "mape_test_score": mape_test_score}) #log the scores
        
baseline_model_samsung()

**XIAOMI**

In [ ]:
def baseline_model_xiaomi():

    with wandb.init(project="DSS") as r:
         
        
        # the last version of the Xiaomi dataset is used
        artifact = r.use_artifact('metu_datacraft/DSS/Xiaomi:v2', type='Data')
        table = artifact.get('weekday_data')
        dataset= {"Date": table.get_column("Date"),"Price":table.get_column("Price"),"Open":table.get_column("Open"),
                  "High":table.get_column("High"), "Low":table.get_column("Low") , "Volume": table.get_column("Volume"),
                  "Change":table.get_column("Change")}

        data = pd.DataFrame(dataset)
        preprocessed_data=preprocess_baseline(data)
        mse_test_score,mape_test_score=baseline_model_create(preprocessed_data)

        wandb.init(project="DSS",job_type="baseline-model", name="baseline_model_XIAOMI")
        wandb.log({"mse_test_score": mse_test_score, "mape_test_score": mape_test_score}) #log the scores
        
baseline_model_xiaomi()